<a href="https://colab.research.google.com/github/HeribertoGodoy/Tareas-EIE-401-/blob/main/Proyectos/Proyecto%202/Proyecto_2_GodoyHeriberto_203197551.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center>

</center></h1>
<font size="6"><center>
EIE 401
PROCESAMIENTO DIGITAL MULTIMEDIA
</center></font>


<center><h2>Proyecto 2</h2></center>
<center><h3>Procesamiento digital de señales</h3></center>
<center><h3>Profesor: Jorge Cardenas</h3></center>

<center><h3>Por: Heriberto Godoy</h3></center>
<center><h5>Pontificia Universidad Catolica de Valparaiso</h5></center>


In [ ]:
from IPython.display import IFrame, display
#filepath = "Proyecto_2_EIE401.pdf"
#IFrame(filepath, width=700, height=400)

En la dirección https://huggingface.co/datasets/astroboy1/metasurfaces_V1_30-90GHZ encontrarás el archivo batch_june2024.zip que contiene las imagenes que usarás para el procesamiento. ATENCION: usa el archivo zip indicado pues tienes las imagenes preparadas para tu trabajo.

## 1. Carga de Datos


- En los archivos, encontrarás un folder con imagenes de 3 categorías: cross, box, circle.
Cada imagen tiene una medida de 512x512 pixeles y 3 canales. Se han acentuado los azules y rojos para facilitar el proceso de obtención de características.

- Puedes extraer la clase a la que pertenece la imagen, desde el nombre del archivo. Ahí mismo encuentras números de serie que te permiten identificar de forma específica una imagen.

- Se te entrega 3 imagenes de referencia de 32x32 pixeles, las que debes usar como plantilla para el proceso de clasificación de la imagen.


In [1]:
!git clone https://github.com/Jorgecardenas1/EIE_401_MULTIMEDIA.git

Cloning into 'EIE_401_MULTIMEDIA'...
remote: Enumerating objects: 347, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 347 (delta 1), reused 1 (delta 0), pack-reused 341
Receiving objects: 100% (347/347), 44.47 MiB | 12.24 MiB/s, done.
Resolving deltas: 100% (159/159), done.


In [3]:
from zipfile import ZipFile

with ZipFile('/content/EIE_401_MULTIMEDIA/Proyectos/Proyecto 2/imagenes.zip', 'r') as zipObj:
   zipObj.extractall('/content/batch_june2024')

## 2. Transformar imagen
Recuerda que la tarea es lograr el procesamiento más rápido posible.
Para lograr esto, realiza las operaciones que consideres necesarias sobre la imagen original para el posterior trabajo de Matching con las plantillas dadas. Crees que trabajar en el dominio de fourier o wavelet ayudaría en las etapas posteriores?

In [4]:
import os
import cv2
import numpy as np
import pywt
from glob import glob
import multiprocessing as mp

# Ruta a las imágenes
ruta_imagenes = '/content/batch_june2024/processed512/*.png'
# Ruta a la carpeta de salida
carpeta_salida = '/content/batch_june2024/transformadas'

# Crear la carpeta de salida si no existe
os.makedirs(carpeta_salida, exist_ok=True)

# Función para procesar una imagen: carga, escala de grises, redimensiona y aplica wavelet
def procesar_imagen(imagen_path):
    try:
        # Cargar la imagen en escala de grises
        imagen = cv2.imread(imagen_path, cv2.IMREAD_GRAYSCALE)

        # Verificar si la imagen se cargó correctamente
        if imagen is None:
            raise ValueError(f"Error al cargar la imagen: {imagen_path}")

        # Redimensionar la imagen
        imagen_redimensionada = cv2.resize(imagen, (128, 128))

        # Aplicar la transformada Wavelet
        coeficientes = pywt.wavedec2(imagen_redimensionada, 'haar', level=2)

        # Reconstruir la imagen con los coeficientes de aproximación (nivel más bajo)
        aproximacion = coeficientes[0]

        # Obtener el nombre de la imagen original
        nombre_imagen = os.path.basename(imagen_path)

        # Guardar la imagen transformada
        cv2.imwrite(os.path.join(carpeta_salida, nombre_imagen), aproximacion)

        return nombre_imagen
    except Exception as e:
        print(f"Error procesando la imagen {imagen_path}: {e}")
        return None

# Obtener rutas de imágenes
imagenes_paths = glob(ruta_imagenes)

# Crear un pool de procesos
with mp.Pool(processes=mp.cpu_count()) as pool:
    # Procesar las imágenes en paralelo
    imagenes_procesadas = pool.map(procesar_imagen, imagenes_paths)

# Filtrar las imágenes que se procesaron correctamente
imagenes_procesadas = [img for img in imagenes_procesadas if img is not None]

print(f"Imágenes procesadas y guardadas en {carpeta_salida}")



Imágenes procesadas y guardadas en /content/batch_june2024/transformadas


## 3. Operaciones
Para extraer las caracteristicas, debes realizar las operaciones de segmentación, normalización o convolución como creas necesarias.
El resultado de tus operaciones vas a usarlo para el proceso de matching.


In [14]:
import os
import cv2
import glob

# Ruta de la carpeta con las imágenes transformadas
carpeta_transformadas = '/content/batch_june2024/transformadas/*.png'

# Ruta de la carpeta donde guardar las imágenes umbralizadas
carpeta_segmentacion = '/content/batch_june2024/segmentacion'

# Crear la carpeta de segmentación si no existe
os.makedirs(carpeta_segmentacion, exist_ok=True)

# Obtener lista de rutas de archivos PNG en la carpeta de transformadas
archivos = glob.glob(carpeta_transformadas)

# Iterar sobre cada archivo en la lista
for ruta_entrada in archivos:
    try:
        # Obtener el nombre de archivo de la ruta completa
        archivo = os.path.basename(ruta_entrada)

        # Cargar la imagen en escala de grises
        imagen = cv2.imread(ruta_entrada, cv2.IMREAD_GRAYSCALE)

        # Verificar si la imagen se cargó correctamente
        if imagen is None:
            raise ValueError(f"Error al cargar la imagen: {ruta_entrada}")

        # Aplicar umbralización
        _, imagen_umbralizada = cv2.threshold(imagen, 127, 255, cv2.THRESH_BINARY)

        # Construir la ruta de salida para la imagen umbralizada
        ruta_salida = os.path.join(carpeta_segmentacion, archivo)

        # Guardar la imagen umbralizada en la carpeta de salida
        cv2.imwrite(ruta_salida, imagen_umbralizada)


    except Exception as e:
        print(f"Error procesando la imagen {ruta_entrada}: {e}")

print("Proceso completado.")





Se truncaron las últimas líneas 5000 del resultado de transmisión.
Imagen umbralizada guardada: /content/batch_june2024/segmentacion/circ_01_freq_reflect_bd00bd23-e53b-11ee-a555-047c16a08772_0-429_80-90.png
Imagen umbralizada guardada: /content/batch_june2024/segmentacion/cross_01_freq_reflect_3be7fd33-bd16-11ee-be43-047c16a08772_0-896_80-90.png
Imagen umbralizada guardada: /content/batch_june2024/segmentacion/cross_01_freq_reflect_3be7fd33-bd16-11ee-be43-047c16a08772_0-1114_80-90.png
Imagen umbralizada guardada: /content/batch_june2024/segmentacion/cross_01_freq_reflect_b1de6052-eca2-11ee-9194-047c16a08772_0-367_80-90.png
Imagen umbralizada guardada: /content/batch_june2024/segmentacion/cross_01_freq_reflect_3be7fd33-bd16-11ee-be43-047c16a08772_0-578_80-90.png
Imagen umbralizada guardada: /content/batch_june2024/segmentacion/cross_01_freq_reflect_3be7fd33-bd16-11ee-be43-047c16a08772_0-519_80-90.png
Imagen umbralizada guardada: /content/batch_june2024/segmentacion/cross_01_freq_reflect

## 4. Matching
El proceso de matching debe considerar que vas a usar las imagenes de referencia para calcular la correlación

![image.png](attachment:6850d57c-7e6f-4acf-ac3b-d9a8feb071cb.png)

Este proceso es similar a un windowing, donde la ventana es la imagen de referencia.
Como verás es un proceso iterativo, dispendioso, pero en el que puedes poner tu creatividad a funcionar para lograrlo de forma eficiente.

#### _TIP: realiza el proceso un batch pequeño de imagenes, o solo sobre un par de ellas, hasta que estés satisfecho con el resultado. Una vez esté listo, ESCALA el procedimiento para operar todas la imagenes lo más rápido posible.!_

In [24]:
import cv2
import os

# Rutas de las plantillas
box_template_path = '/content/EIE_401_MULTIMEDIA/Proyectos/Proyecto 2/box_template.png'
circ_template_path = '/content/EIE_401_MULTIMEDIA/Proyectos/Proyecto 2/circ_template.png'
cross_template_path = '/content/EIE_401_MULTIMEDIA/Proyectos/Proyecto 2/cross_template.png'

# Ruta de la carpeta con las imágenes de características
images_path = '/content/batch_june2024/segmentacion'

# Ruta de la carpeta para almacenar los resultados
results_path = '/content/results'
os.makedirs(results_path, exist_ok=True)

# Imprimir las rutas de las plantillas para verificación
print(f'Ruta box_template: {box_template_path}')
print(f'Ruta circ_template: {circ_template_path}')
print(f'Ruta cross_template: {cross_template_path}')

# Cargar las plantillas
box_template = cv2.imread(box_template_path, cv2.IMREAD_GRAYSCALE)
circ_template = cv2.imread(circ_template_path, cv2.IMREAD_GRAYSCALE)
cross_template = cv2.imread(cross_template_path, cv2.IMREAD_GRAYSCALE)

# Verificar que las plantillas se hayan cargado correctamente
if box_template is None:
    print(f'Error al cargar box_template desde: {box_template_path}')
if circ_template is None:
    print(f'Error al cargar circ_template desde: {circ_template_path}')
if cross_template is None:
    print(f'Error al cargar cross_template desde: {cross_template_path}')
if box_template is None or circ_template is None or cross_template is None:
    raise ValueError("Una o más plantillas no se pudieron cargar. Verifica las rutas.")

templates = {
    'box': box_template,
    'circ': circ_template,
    'cross': cross_template
}

# Función para realizar la coincidencia de plantillas
def match_template(image, template):
    result = cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
    return max_val

# Procesar cada imagen en la carpeta
for image_name in os.listdir(images_path):
    image_path = os.path.join(images_path, image_name)
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Verificar que la imagen se haya cargado correctamente
    if image is None:
        print(f'No se pudo cargar la imagen: {image_path}')
        continue

    # Verificar que la imagen tenga el tamaño correcto
    if image.shape[0] < 16 or image.shape[1] < 16:
        print(f'La imagen {image_path} es más pequeña que las plantillas.')
        continue

    best_match = None
    best_score = -1

    for template_name, template in templates.items():
        score = match_template(image, template)
        if score > best_score:
            best_score = score
            best_match = template_name

    result_image_path = os.path.join(results_path, f'{best_match}_{image_name}')
    cv2.imwrite(result_image_path, image)

    print(f'Imagen: {image_name}, Mejor coincidencia: {best_match}, Puntaje: {best_score}')



Ruta box_template: /content/EIE_401_MULTIMEDIA/Proyectos/Proyecto 2/box_template.png
Ruta circ_template: /content/EIE_401_MULTIMEDIA/Proyectos/Proyecto 2/circ_template.png
Ruta cross_template: /content/EIE_401_MULTIMEDIA/Proyectos/Proyecto 2/cross_template.png
Error al cargar box_template desde: /content/EIE_401_MULTIMEDIA/Proyectos/Proyecto 2/box_template.png
Error al cargar circ_template desde: /content/EIE_401_MULTIMEDIA/Proyectos/Proyecto 2/circ_template.png
Error al cargar cross_template desde: /content/EIE_401_MULTIMEDIA/Proyectos/Proyecto 2/cross_template.png


ValueError: Una o más plantillas no se pudieron cargar. Verifica las rutas.

In [19]:
import os
import shutil

# Ruta de la carpeta que deseas borrar
carpeta_a_borrar = '/content/batch_june2024/matching_results'

# Verificar si la carpeta existe
if os.path.exists(carpeta_a_borrar):
    # Borrar el contenido de la carpeta
    try:
        shutil.rmtree(carpeta_a_borrar)
        print(f"Carpeta {carpeta_a_borrar} y su contenido han sido eliminados correctamente.")
    except OSError as e:
        print(f"Error al intentar eliminar la carpeta {carpeta_a_borrar}: {e}")
else:
    print(f"La carpeta {carpeta_a_borrar} no existe.")


Carpeta /content/batch_june2024/matching_results y su contenido han sido eliminados correctamente.


# 5. Clasificación
De cada operación realizada de matching, obtendrás valores para inferir si la imagen bajo estudio corresponde con alguna de las 3 categorías: cross, box, circle.

Debes almacenar cada resultado obtenido para cada imagen, hallar el error de clasificación y generar un informe del error general al clasificar todas las imagenes.

Presenta aquí el resultado de clasificación numérico, así como  dos ejemplos verdaderos y dos ejemplos falsos de imagenes clasificadas.


In [18]:
import os

# Ruta de la carpeta con los resultados de matching
carpeta_matching = '/content/batch_june2024/matching_results'

# Diccionario para almacenar los resultados de clasificación
resultados_clasificacion = {}

# Leer los resultados de matching
for file_name in os.listdir(carpeta_matching):
    if file_name.endswith('.txt'):
        # Obtener el nombre de la imagen
        nombre_imagen = os.path.splitext(file_name)[0]

        # Leer el archivo de resultados
        with open(os.path.join(carpeta_matching, file_name), 'r') as f:
            lines = f.readlines()
            for line in lines:
                if line.startswith('Template:'):
                    partes = line.strip().split(',')
                    template_name = partes[0].split(': ')[1].strip()
                    max_corr = float(partes[1].split(': ')[1].strip())
                    # Guardar el resultado de clasificación
                    resultados_clasificacion[nombre_imagen] = (template_name, max_corr)

# Definir las categorías verdaderas de cada imagen según el nombre
categorias_reales = {
    'ejemplo_verdadero_1': 'box',
    'ejemplo_verdadero_2': 'circ',
    'ejemplo_falso_1': 'cross',
    'ejemplo_falso_2': 'box'
}

# Calcular el error de clasificación
errores = []
aciertos = 0

for imagen, (prediccion, _) in resultados_clasificacion.items():
    if imagen in categorias_reales:
        categoria_real = categorias_reales[imagen]
        if prediccion == categoria_real:
            aciertos += 1
        else:
            errores.append((imagen, categoria_real, prediccion))
    else:
        print(f"Error: No se encontró la categoría real para la imagen {imagen}")

# Calcular métricas de error
total_imagenes = len(resultados_clasificacion)
exactitud = aciertos / total_imagenes if total_imagenes > 0 else 0
error_general = 1 - exactitud

# Generar informe de error general
print(f"--- Informe de Error de Clasificación ---")
print(f"Total de imágenes: {total_imagenes}")
print(f"Aciertos: {aciertos}")
print(f"Errores: {len(errores)}")
print(f"Exactitud: {exactitud:.2f}")
print(f"Error general: {error_general:.2f}")

# Mostrar ejemplos de clasificación solo si hay errores
if len(errores) > 0:
    print("\n--- Ejemplos de Clasificación ---")
    print("Ejemplos Verdaderos:")
    for i in range(min(2, len(errores))):
        imagen, categoria_real, prediccion = errores[i]
        print(f"Imagen: {imagen}, Real: {categoria_real}, Predicción: {prediccion}")

    print("\nEjemplos Falsos:")
    for i in range(max(0, len(errores)-2), len(errores)):
        imagen, categoria_real, prediccion = errores[i]
        print(f"Imagen: {imagen}, Real: {categoria_real}, Predicción: {prediccion}")
else:
    print("\nNo se encontraron errores de clasificación.")




Se truncaron las últimas líneas 5000 del resultado de transmisión.
Error: No se encontró la categoría real para la imagen circ_01_freq_reflect_bd00bd23-e53b-11ee-a555-047c16a08772_0-339_80-90.png_matching
Error: No se encontró la categoría real para la imagen circ_01_freq_reflect_6d743d30-e161-11ee-907c-047c16a08772_0-160_80-90.png_matching
Error: No se encontró la categoría real para la imagen cross_01_freq_reflect_b1de6052-eca2-11ee-9194-047c16a08772_0-881_80-90.png_matching
Error: No se encontró la categoría real para la imagen box_01_freq_reflect_e3744294-aba9-11ee-b4d0-047c16a08772_0-497_80-90.png_matching
Error: No se encontró la categoría real para la imagen circ_01_freq_reflect_536ce6b7-e07b-11ee-9fac-047c16a08772_0-66_80-90.png_matching
Error: No se encontró la categoría real para la imagen circ_01_freq_reflect_6d743d30-e161-11ee-907c-047c16a08772_0-420_80-90.png_matching
Error: No se encontró la categoría real para la imagen cross_01_freq_reflect_7357cade-0e1a-11ef-859e-047c1

# 6. Medición
Recuerda que debes imprimir el tiempo que tarda tu algoritmo en procesar todas las imagenes en la base de datos. Usa el siguiente fragmento de código, ubicando el inicio y fin donde corresponda.





In [ ]:
from timeit import default_timer as timer

start = timer()

print(23*2.3)

end = timer()
print(end - start)

